In [1]:
%matplotlib inline
import pandas as pd
import datetime
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.utils.data.dataloader import default_collate
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from torch.autograd import Variable
print(torch.__version__)

1.0.0


In [2]:
class IMUDataset(Dataset):
    def __init__(self, mode = 'test', transform = None):
        if mode == 'train' :
            self.df = pd.read_csv('data_preparation/train.csv', header = 0, index_col = 0)
        elif mode == 'test' :
            self.df = pd.read_csv('data_preparation/test.csv', header = 0, index_col = 0)
        elif mode == 'val' :
            self.df = pd.read_csv('data_preparation/val.csv', header = 0, index_col = 0)
        self.transform = transform
        self.df = self.df.reset_index()
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        y = self.df.iloc[idx : idx + 80, 3 : ].values
        ind = np.argmax(np.sum(y, axis = 0))
        label = np.zeros_like(self.df.iloc[0, 3 : ].values)
        label = label.astype('float')
        label[ind] = 1
        x = self.df.iloc[idx : idx + 80, : 3].values
        x = x.astype('float')
        assert(x.shape == (80, 3))
        assert(label.shape == (6, ))
        return x, label
        
trainset = IMUDataset(mode = 'train')
valset = IMUDataset(mode = 'val')
testset = IMUDataset(mode = 'test')

In [3]:
train_batch_size = 10240
batch_size = 256
def my_collate(batch):
    "Puts each data field into a tensor with outer dimension batch size"
    batch = list(filter(lambda x : x is not None, batch))
    return default_collate(batch)

# train_indices = [(i * 80) for i in range(len(trainset) // 80)]
# val_indices = [(i * 80) for i in range(len(valset) // 80)]
# test_indices = [(i * 80) for i in range(len(testset) // 80)]

# trainloader = DataLoader(trainset, batch_size = batch_size, sampler = SubsetRandomSampler(train_indices), collate_fn = my_collate, drop_last = True)
# valloader = DataLoader(valset, batch_size = batch_size, sampler = SubsetRandomSampler(val_indices), collate_fn = my_collate, drop_last = True)
# testloader = DataLoader(testset, batch_size = batch_size, sampler = SubsetRandomSampler(test_indices), collate_fn = my_collate, drop_last = True)

train_indices = [i for i in range(len(trainset) - 80)]
val_indices = [i for i in range(len(valset) - 80)]
test_indices = [i for i in range(len(testset) - 80)]

trainloader = DataLoader(trainset, batch_size = train_batch_size, sampler = SubsetRandomSampler(train_indices), collate_fn = my_collate, drop_last = True)
valloader = DataLoader(valset, batch_size = batch_size, sampler = SubsetRandomSampler(val_indices), collate_fn = my_collate, drop_last = True)
testloader = DataLoader(testset, batch_size = batch_size, sampler = SubsetRandomSampler(test_indices), collate_fn = my_collate, drop_last = True)

# for idx, (x, y) in enumerate(trainloader) :
#     print(x.shape, y.shape)

In [10]:
class ConvAutoEncoder(nn.Module):
    def __init__(self, code_size):
        super(ConvAutoEncoder, self).__init__()
        self.code_size = code_size
        # defining layers
        # input size - batch_size x 3 x 100
        conv1 = nn.Conv1d(3, 10, 3)
        # after conv1 - batch_size x 10 x 98
        conv2 = nn.Conv1d(10, 20, 1)
        # after conv2 - batch_size x 10 x 98
        enc_fc = nn.Linear(10 * 98, self.code_size)
        dec_fc = nn.Linear(self.code_size, 10 * 98)
        deconv2 = nn.ConvTranspose1d(20, 10, 1)
        deconv1 = nn.ConvTranspose1d(10, 3, 3)
        
    def forward(self, signal):
        code = self.encode(signal)
        out = self.decode(code)
        return out, code
    
    def encode(signal):
        signal = signal.view(16, 3, 80)
        code = F.sigmoid(self.conv1(signal))
        code = F.sigmoid(self.conv2(code))
        code = F.sigmoid(self.enc_fc(code))
        return code
    
    def decode(code):
        out = F.sigmoid(self.dec_fc(code))
        out = F.sigmoid(self.deconv2(out))
        out = F.sigmoid(self.deconv1(out))
        return out
    
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # defining layers
        self.conv1 = nn.Conv1d(3, 5, 3)
        self.conv2 = nn.Conv1d(5, 10, 5)
        self.conv3 = nn.Conv1d(10, 10, 5)
        self.fc = nn.Linear(70 * 10, 6)
#         self.dropout = nn.Dropout(0.25)
#         self.bn = nn.BatchNorm1d(5)
        
        nn.init.xavier_uniform_(self.conv1.weight, gain = nn.init.calculate_gain('relu'))
        nn.init.xavier_uniform_(self.conv2.weight, gain = nn.init.calculate_gain('relu'))
        nn.init.xavier_uniform_(self.conv3.weight, gain = nn.init.calculate_gain('relu'))
        
    def forward(self, signal):
        signal = signal.view(-1, 3, 80)
        out = F.relu(self.conv1(signal))
        out = self.dropout(out)
        out = F.relu(self.conv2(out))
        out = self.dropout(out)
        out = F.relu(self.conv3(out))
        out = self.dropout(out)
        out = out.view(-1, 70 * 10)
        out = self.fc(out)
        out = self.dropout(out)
        out = F.log_softmax(out, dim = 1)
        return out
    
Net = ConvNet()
if torch.cuda.is_available():
    print('Model on GPU')
    Net = Net.cuda()
print(Net.state_dict())

Model on GPU
OrderedDict([('conv1.weight', tensor([[[-0.2830,  0.2197,  0.1698],
         [-0.0961, -0.1935, -0.3161],
         [-0.1865,  0.0332, -0.1903]],

        [[-0.3178,  0.1296, -0.0465],
         [-0.1400, -0.2655,  0.3078],
         [ 0.0539, -0.1435, -0.1577]],

        [[-0.0438, -0.1054, -0.3167],
         [ 0.2678,  0.1481,  0.2358],
         [ 0.2440,  0.3309,  0.0138]],

        [[-0.2408, -0.2398, -0.2068],
         [ 0.0850, -0.0181, -0.1646],
         [ 0.3071,  0.1067,  0.2089]],

        [[-0.1302,  0.0411, -0.1165],
         [-0.0448, -0.1281, -0.2045],
         [ 0.2050, -0.1095,  0.2455]]], device='cuda:0')), ('conv1.bias', tensor([-0.3259, -0.3108, -0.1264,  0.1304,  0.1348], device='cuda:0')), ('conv2.weight', tensor([[[ 0.0504,  0.1638, -0.0790,  0.0030,  0.0485],
         [-0.1787,  0.0898,  0.1914,  0.0397, -0.1790],
         [ 0.1860, -0.0476, -0.1375, -0.0893, -0.1310],
         [-0.1631, -0.1983,  0.0250,  0.0937,  0.1162],
         [-0.1044,  0.0039,  

In [5]:
def output_size(n, f, p = 0, s = 1):
    return (((n + 2 * p - f) / s) + 1)

print(output_size(80, 3))
print(output_size(78, 5))
print(output_size(74, 5))

78.0
74.0
70.0


In [6]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Net.parameters(), lr=1e-4)

In [24]:
num_epochs = 30
total_step = len(trainset) // train_batch_size
train_loss_list = list()
val_loss_list = list()
min_val = 100
for epoch in range(num_epochs):
    trn = []
    Net.train()
    for i, (images, labels) in enumerate(trainloader) :
        images = Variable(images).cuda().float()
        labels = Variable(labels).cuda()
        _, target = torch.max(labels, 1)

        y_pred = Net(images)
        
        loss = criterion(y_pred, target)
        trn.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_value_(Net.parameters(), 10)
        optimizer.step()

#         clear_output(wait=True)
#         print(f'epoch: {epoch}, step : {i * len(images)} of {total_step}, loss : {loss}')
        if i % 20 == 0 :
            print('epoch = ', epoch, ' step = ', i, ' of total steps ', total_step, ' loss = ', loss.item())
    train_loss = (sum(trn) / len(trn))
    train_loss_list.append(train_loss)
    
    Net.eval()
    val = []
    with torch.no_grad() :
        for i, (images, labels) in enumerate(valloader) :
            images = Variable(images).cuda().float()
            labels = Variable(labels).cuda()
            _, target = torch.max(labels, 1)

            # Forward pass
            outputs = Net(images)

#             loss = criterion(outputs.float(), labels.float())
            loss = criterion(outputs, target)
            val.append(loss)
#     print(f'validation loss : {sum(val)/len(val)}')
    val_loss = (sum(val) / len(val)).item()
    val_loss_list.append(val_loss)
    print('epoch : ', epoch, ' / ', num_epochs, ' | TL : ', train_loss, ' | VL : ', val_loss)
    
    if val_loss < min_val :
        print('saving model')
        min_val = val_loss
        torch.save(Net.state_dict(), 'model_dropout.pt')

epoch =  0  step =  0  of total steps  76  loss =  1.7922731637954712


KeyboardInterrupt: 

In [5]:
Net = ConvNet().eval()
Net.load_state_dict(torch.load('model_dropout5.pt'))
print(Net.state_dict())

OrderedDict([('conv1.weight', tensor([[[-0.2538,  0.4009, -0.4006],
         [ 0.2252, -0.1528, -0.6841],
         [-0.2669, -0.1826,  0.4054]],

        [[ 0.6621, -0.4298, -0.4978],
         [-0.2886, -0.3838,  0.0866],
         [ 0.5184,  0.6247, -0.6648]],

        [[ 0.5460,  0.6890, -0.0698],
         [ 0.1503, -0.0947, -0.2529],
         [-0.1997,  0.2127,  0.0085]],

        [[-0.3807,  0.0490,  0.1034],
         [ 0.1895, -0.6926, -0.1681],
         [ 0.3082,  0.5064,  0.2634]],

        [[ 0.0828, -0.4887,  0.5240],
         [ 0.4208, -0.2635, -0.0619],
         [ 0.5133, -0.1343, -0.1466]]])), ('conv1.bias', tensor([-0.3190, -0.0330,  0.0064,  0.1227,  0.2773])), ('conv2.weight', tensor([[[ 0.1485,  0.2653,  0.2102, -0.2603,  0.2093],
         [-0.2782, -0.1281,  0.2997,  0.3013,  0.3720],
         [-0.3849,  0.0593, -0.3018,  0.0773, -0.1321],
         [ 0.2473, -0.1473,  0.1716,  0.0083,  0.1994],
         [-0.2193, -0.3334, -0.0741,  0.2403, -0.3519]],

        [[-0.0705,

In [9]:
def _get_accuracy(dataloader):
    total = 0
    correct = 0
    Net.eval()
    for i, (images, labels) in enumerate(dataloader):
        images = Variable(images).float()
        labels = Variable(labels).float()
#         print(images)
        outputs = Net(images)
        print('Output : ')
        print(outputs)
        
        _, label_ind = torch.max(labels, 1)
        _, pred_ind = torch.max(outputs, 1)
        
        # converting to numpy arrays
        label_ind = label_ind.data.numpy()
        pred_ind = pred_ind.data.numpy()
        
        print('Label : ')
        print(label_ind)
        print('Prediction : ')
        print(pred_ind)
        
        # get difference
        diff_ind = label_ind - pred_ind
        # correctly classified will be 1 and will get added
        # incorrectly classified will be 0
        correct += np.count_nonzero(diff_ind == 0)
        total += len(diff_ind)

        print('correct - ', correct, ' of ', total, 'as of now')
        print('image - ', total, ' of ', len(dataloader.dataset) // 80)

    accuracy = correct / total
    # print(len(diff_ind))
    return accuracy

Net.cpu()
_get_accuracy(trainloader)

Output : 
tensor([[-2.0044, -1.3722, -2.5353, -2.4691, -1.9442, -1.1884],
        [-1.9215, -1.5258, -2.5783, -2.4415, -1.8602, -1.1469],
        [-2.0818, -1.2835, -2.6184, -2.6120, -1.8854, -1.2035],
        ...,
        [-2.0583, -1.2906, -2.4684, -2.5378, -1.9714, -1.2235],
        [-2.0492, -1.3299, -2.6497, -2.5191, -1.9292, -1.1706],
        [-1.9996, -1.4634, -2.6320, -2.4647, -1.9628, -1.0915]],
       grad_fn=<LogSoftmaxBackward>)
Label : 
[4 0 1 ... 5 1 1]
Prediction : 
[5 5 5 ... 5 5 5]
correct -  4245  of  10240 as of now
image -  10240  of  9792
Output : 
tensor([[-1.8735, -1.4578, -2.5437, -2.4474, -1.9167, -1.1990],
        [-1.9541, -1.4405, -2.7849, -2.5840, -2.0011, -1.0524],
        [-2.0160, -1.4973, -2.6175, -2.4708, -1.8480, -1.1146],
        ...,
        [-2.0963, -1.4071, -2.5779, -2.4293, -1.8281, -1.1793],
        [-1.9645, -1.4715, -2.5973, -2.4809, -1.9244, -1.1206],
        [-1.9976, -1.3889, -2.5708, -2.5757, -1.8871, -1.1683]],
       grad_fn=<LogSoftmax

KeyboardInterrupt: 